# Interactive Plots

This notebook is used to generate interactive visualizations with the python plotly library for use in the online documentation.

In [1]:
import numpy as np

import plotly
import plotly.express as px
import plotly.graph_objects as go
from itertools import cycle

from plot_funcs import (get_results, get_mean_avg_ranks, get_ranks_sizes,
                        get_highest_performing_df, get_cut_off_df,
                        get_across_ranks, get_intra_pipeline_df,
                        get_single_vs_multiple_df, get_results_df,
                        get_single_vs_multiple_df)

In [2]:
results = get_results('../exp/results')

static = {'random': True, 'base': True, 'fs': True, 'ico': True}
ensemble_only = {'stacked': True, 'voted': True}
all_parcels = {**static, **ensemble_only, 'grid': True}

params = dict(results=results, keep_full_name=True, threshold=False, **static)

ts = [True, True, True, True]
fs = [False, False, False, False]

Found: 64530 Incomplete: 0


In [3]:
def init_fig():
    
    fig = go.Figure()
    fig.layout.xaxis.title = 'Size'
    fig.layout.yaxis.title = 'Mean Rank'
    
    return fig

def add_base_side_annot(fig, buttons):
    
    # Add dropdown
    fig.update_layout(
        updatemenus=[
            dict(
                type = "buttons",
                direction = "left",
                buttons=buttons,
                pad={"r": 10, "t": 10},
                showactive=True,
                x=0.11,
                xanchor="left",
                y=1.1,
                yanchor="top"
            ),
        ]
    )

    # Add annotation
    fig.update_layout(
        annotations=[
            dict(text="Scale:", showarrow=False,
                 x=0, y=1.08, yref="paper", align="left")
        ]
    )

def add_go_base_scatter(fig, name, df, visible, marker_size=20):
    
    gs = go.Scatter(
            x=df['Size'],
            y=df['Mean_Rank'],
            visible=visible,
            mode='markers',
            name=name,
            hovertext=df['full_name'],
            textposition="middle center",
            customdata=np.stack((df['r2'],
                                 df['roc_auc'],
                                 df['rank_label'],
                                 df['size_label']), axis=-1),
            hovertemplate='<b>%{hovertext}</b><br>' +
                          '%{customdata[2]}<br>' +
                          '%{customdata[3]}<br>' +
                          'Mean R2: %{customdata[0]:.4f}<br>' +
                          'Mean ROC AUC: %{customdata[1]:.4f}<br>' +
                          '<extra></extra>')

    gs.marker.size = marker_size
    gs.marker.opacity = .5

    # Add plot
    fig.add_trace(gs)
    
def add_go_by_target_scatter(fig, name, df, visible, palette, marker_size=20):
    
    gs = go.Scatter(
            x=df['Size'],
            y=df['Mean_Rank'],
            visible=visible,
            mode='markers',
            name=name,
            text=df['full_name'],
            customdata=np.stack((df['rank_label'],
                                 df['size_label'],
                                 df['Mean_Score']), axis=-1),
            hovertemplate='<b>%{text}</b><br>' +
                          '<b>' + name + '</b><br>' + 
                          '%{customdata[0]}<br>' +
                          '%{customdata[1]}<br>' +
                          'Mean Raw Metric: %{customdata[2]:.4f}<extra></extra>')

    gs.marker.size = marker_size
    gs.marker.opacity = .25
    gs.marker.color = next(palette)

    # Add plot
    fig.add_trace(gs)
    
def by_pipeline_base(marker_size=20, xrange=(.5, 4), yrange=(1.2, 2.8), **parcels):
    
    models = ['LGBM', 'SVM', 'Elastic-Net', 'All']

    fig_dict = {
        "data": [],
        "layout": {},
        "frames": []
    }

    fig_dict["layout"]["xaxis"] = {"range": xrange, "title": "Size (log10)"}
    fig_dict["layout"]["yaxis"] = {"range": yrange, "title": "Mean Rank (log10)"}
    fig_dict["layout"]["hovermode"] = "closest"

    sliders_dict = {
        "active": 0,
        "yanchor": "top",
        "xanchor": "left",
        "currentvalue": {
            "font": {"size": 20},
            "prefix": "Pipeline:",
            "visible": True,
            "xanchor": "right"
        },
        "pad": {"b": 10, "t": 50},
        "len": 0.9,
        "x": 0.1,
        "y": 0,
        "steps": []
    }

    def get_data_dicts(pipeline):

        if pipeline == 'All':
            models = ['elastic', 'lgbm', 'svm']
        elif pipeline == 'Elastic-Net':
            models = ['elastic']
        elif pipeline == 'LGBM':
            models = ['lgbm']
        else:
            models = ['svm']

        r_df = get_ranks_sizes(results=results,
                               keep_full_name=True, threshold=False,
                               log=True, add_ranks_labels=True,
                               add_raw=True, models=models, **parcels)

        data_dicts = []
        for name, df in r_df.groupby('Parcellation_Type'):
            data_dict = {
                'x': df['Size'],
                'y': df['Mean_Rank'],
                'mode': "markers",
                'text': df['full_name'],
                "marker": {
                    "size": marker_size,
                    "opacity": .5
                },
                "customdata": np.stack((df['r2'],
                                        df['roc_auc'],
                                        df['rank_label'],
                                        df['size_label']), axis=-1),
                'name': name,
                'hovertemplate': '<b>%{text}</b><br>' +
                                 'Mean Rank: %{y:.3f}<br>' +
                                 '%{customdata[2]}<br>' +
                                 '%{customdata[3]}<br>' +
                                 'Mean R2: %{customdata[0]:.4f}<br>' +
                                 'Mean ROC AUC: %{customdata[1]:.4f}<br>' +
                                 '<extra></extra>'
            }

            data_dicts.append(data_dict)

        return data_dicts

    # make init data
    fig_dict["data"] = get_data_dicts(pipeline='LGBM')

    # make frames
    for model in models:
        frame = {"data": [], "name": str(model)}

        frame['data'] = get_data_dicts(model)
        fig_dict["frames"].append(frame)

        slider_step = {"args": [
            [model],
            {"frame": {"duration": 1000, "redraw": False},
             "transition": {"duration": 1000}}
        ],
            "label": model,
            "method": "animate"}

        sliders_dict["steps"].append(slider_step)

    fig_dict["layout"]["sliders"] = [sliders_dict]
    fig = go.Figure(fig_dict)
    
    return fig

## Base Single Parcellations -  log toggle

In [4]:
def gen_single_parcel_fig(marker_size=20):
    
    fig = init_fig()

    def add_traces(r_df, visible=False):
        for name, df in r_df.groupby('Parcellation_Type'):
            add_go_base_scatter(fig, name, df, visible, marker_size=marker_size)

    # Keep track of buttons
    buttons = []

    # Base
    add_traces(get_ranks_sizes(**params, add_raw=True,
                               log=False, add_ranks_labels=True), visible=True)
    buttons.append(dict(args=[{"visible": ts + fs}],
                        label='Base',
                        method="update"))

    # Log10
    add_traces(get_ranks_sizes(**params, add_raw=True,
                               log=True, add_ranks_labels=True), visible=False)
    buttons.append(dict(args=[{"visible": fs + ts}],
                        label='Log',
                        method="update"))

    # Add base side dropdown + annot
    add_base_side_annot(fig, buttons)
    
    return fig

#plotly.offline.plot(gen_single_parcel_fig(marker_size=20))
gen_single_parcel_fig(marker_size=20).write_html('../docs/interactive1.html')

## Base Single Parcellations - by pipeline

In [5]:
def gen_pipeline_fig(marker_size=20):
    
    return by_pipeline_base(marker_size=marker_size,
                            yrange=(1.25, 2.5),
                            **static)

#plotly.offline.plot(gen_pipeline_fig(marker_size=20))
gen_pipeline_fig(marker_size=10).write_html('../docs/_includes/interactive2.html')
gen_pipeline_fig(marker_size=20).write_html('../docs/interactive2.html')

## By Target - avg over pipeline

In [6]:
def make_target_fig(marker_size=15, **parcels):

    fig = init_fig()
    
    def add_traces(r_df, visible=False):
        
        # Fixed colors
        palette = cycle(px.colors.qualitative.D3)

        # By target here
        for name, df in r_df.groupby('target'):
            add_go_by_target_scatter(fig, name, df, visible,
                                     palette, marker_size=marker_size)

    # Keep track of buttons
    buttons = []

    # Base
    add_traces(get_ranks_sizes(results=results,
                               keep_full_name=True,
                               threshold=False, avg_targets=False,
                               log=False, add_ranks_labels=True,
                               add_raw=False,
                               **parcels), visible=True)
    buttons.append(dict(args=[{"visible": ([True]*45) + ([False]*45)}],
                        label='Base',
                        method="update"))

    # Log10
    add_traces(get_ranks_sizes(results=results, keep_full_name=True, threshold=False,
                               avg_targets=False, log=True,
                               add_ranks_labels=True, add_raw=False,
                               **parcels), visible=False)
    buttons.append(dict(args=[{"visible": ([False]*45) + ([True]*45)}],
                        label='Log',
                        method="update"))

    # Add base side dropdown + annot
    add_base_side_annot(fig, buttons)
    
    return fig

#plotly.offline.plot(make_target_fig(marker_size=15, **static))
make_target_fig(marker_size=15, **static).write_html('../docs/interactive3.html')

## All - base w/ log toggle

In [7]:
def gen_multi_parcel_fig(marker_size=20):
    
    fig = init_fig()

    def add_traces(r_df, visible=False):
        for name, df in r_df.groupby('Parcellation_Type'):
            add_go_base_scatter(fig, name, df, visible, marker_size=marker_size)

    # Keep track of buttons
    buttons = []
    
    # Base
    r_df = get_single_vs_multiple_df(log=False, add_raw=True, add_ranks_labels=True,
                                     stacked=True, voted=True, grid=True,
                                     **params).reset_index()
    add_traces(r_df, visible=True)
    
    buttons.append(dict(args=[{"visible": ts + fs}],
                        label='Base',
                        method="update"))

    # Log10
    r_df = get_single_vs_multiple_df(log=True, add_raw=True, add_ranks_labels=True,
                                     stacked=True, voted=True, grid=True,
                                     **params).reset_index()
    add_traces(r_df, visible=False)

    buttons.append(dict(args=[{"visible": fs + ts}],
                        label='Log',
                        method="update"))

    # Add base side dropdown + annot
    add_base_side_annot(fig, buttons)
    
    return fig

#plotly.offline.plot(gen_multi_parcel_fig(marker_size=20))
gen_multi_parcel_fig(marker_size=20).write_html('../docs/interactive4.html')

## All - by pipeline

In [8]:
def gen_all_pipeline_fig(marker_size=20):
    
    return by_pipeline_base(marker_size=marker_size,
                            yrange=(1.2, 2.8),
                            **all_parcels)

#plotly.offline.plot(gen_all_pipeline_fig(marker_size=20))
gen_all_pipeline_fig(marker_size=10).write_html('../docs/_includes/interactive5.html')
gen_all_pipeline_fig(marker_size=20).write_html('../docs/interactive5.html')

## Ensemble - by pipeline

In [9]:
def gen_ensemble_pipeline_fig(marker_size=20):
    
    return by_pipeline_base(marker_size=marker_size,
                            xrange=(2.25, 4),
                            yrange=(1.2, 2.4),
                            **ensemble_only)

#plotly.offline.plot(gen_ensemble_pipeline_fig(marker_size=20))
gen_ensemble_pipeline_fig(marker_size=10).write_html('../docs/_includes/interactive6.html')
gen_ensemble_pipeline_fig(marker_size=20).write_html('../docs/interactive6.html')

## All By Target - avg over pipeline

In [10]:
#plotly.offline.plot(make_target_fig(marker_size=15, **all_parcels))
make_target_fig(marker_size=15, **all_parcels).write_html('../docs/interactive7.html')